# Planning for the overall mass-ratio estimation code
This notebook is for planning how to do forward modeling to measure the mass-ratio distribution. I need code to do the following:

  - Model the data given certain parameters:
    - Physical Parameters:
      - Overall multiplicity rate
      - mass-ratio distribution $$P(q) \sim q^{\Gamma}$$
      - Seperation distribution - lognormal: $$P(a) = \frac{1}{a\sigma\sqrt{2\pi}}e^{(\ln{a} - \mu)^2 / 2\sigma^2}$$
      - eccentricity distribution - Power law $$P(e) \sim e^{\alpha}$$
    - Observational Parameters
      - Sample bias
        - Malmquist
        - Do I have any other sampling biases?
      - Detection bias
        - Each target has a detection rate as a function of companion temperature
          - The rate estimate comes from 17 samples, so has an uncertainty coming from binomial statistics. Use HelperFunctions.BinomialErrors(n_detected, 17) 
          - The rate also depends on how well we know the primary temperature. Can that be parameterized in some way?
        - separation cutoff: I cannot detect companions more than about 1" from the primary. The exact cutoff is different for each star, but I can probably treat it as just an instrument-specific parameter
      - False positives
        - Astrophysical (i.e. background sources)
        - Other (spurious CCF peaks, etc). This should depend on the number of observations. It also depends on the companion temperature (can that be parameterized?)
        
  - My detections have the following properties
    - temperature estimate, with error bars
      - This can turn into a mass estimate using spectral-type relationships, assuming the companion is MS. That makes adds further uncertainty because of the accuracy of the MS relationship, not to mention the question of whether the companions *are* MS!
      - Going through the primary properties can then give the mass-ratio, with additional uncertainties coming from uncertainty in the primary star properties.
    - $N_{obs}$ measurements of the velocity of both the primary and secondary stars
      - The measurements should not be assumed to be in the same rest frame
      - $N_{obs} >= 1$. If only one measurement, I can't directly measure the mass-ratio
    - $N_{img}$ direct imaging measurements. These do a few things:
      1. If I detect the companion, I get a $\Delta m$ and projected separation measurement.
      2. If I don't I get some form of an upper limit on the separation distribution. (STILL NEED TO FIGURE OUT HOW TO BEST MEASURE THAT! PROBABLY SIGNAL INJECTION/RECOVERY)
      
  - I need to determine the best physical parameters ($\Gamma, \mu, \sigma, \alpha$, multiplicity rate)
    - Given the parameters, I should be able to randomly sample several instances of my experiment.
      - Each should give me N detections, each with some set of properties
    - How to calculate the fitness/likelihood of the given parameters? I need to know how similar my data are to the simulated data, but don't want to go through histograms or something that is binning-specific.
    

# Take 2:
  - Assume the same parameter distributions as above
  - Follow the approach that Adam takes:
    $$f(q,a|F, \Gamma, \mu, \sigma) \sim F \cdot q^{\Gamma} \cdot \frac{1}{a\sigma\sqrt{2\pi}}e^{(\ln{a} - \mu)^2 / 2\sigma^2}$$
    $$P(\Gamma, \mu, \sigma|data) = P(data|\Gamma, \mu, \sigma)P(\Gamma, \mu, \sigma)$$
     - The likelihood function comes from:
       1. Split the parameter space into a fine grid in q, a
       2. For each grid point, find the number of binaries within the grid
       3. For each grid point, find the total number of binaries I am sensitive to
       4. Use poisson (or binomial) statistics to calculate P(number I detect | number of detectable)
       5. The product of the grid probabilities is the likelihood function.